In [55]:
import pandas as pd
df = pd.read_csv('HR Analytics/train.csv')
df.head(2)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0


In [2]:
df.shape

(54808, 14)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           54808 non-null  int64  
 1   department            54808 non-null  object 
 2   region                54808 non-null  object 
 3   education             52399 non-null  object 
 4   gender                54808 non-null  object 
 5   recruitment_channel   54808 non-null  object 
 6   no_of_trainings       54808 non-null  int64  
 7   age                   54808 non-null  int64  
 8   previous_year_rating  50684 non-null  float64
 9   length_of_service     54808 non-null  int64  
 10  KPIs_met >80%         54808 non-null  int64  
 11  awards_won?           54808 non-null  int64  
 12  avg_training_score    54808 non-null  int64  
 13  is_promoted           54808 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 5.9+ MB


In [4]:
df.isna().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [5]:
df['education'].unique()

array(["Master's & above", "Bachelor's", nan, 'Below Secondary'],
      dtype=object)

In [6]:
df['previous_year_rating'].unique()

array([ 5.,  3.,  1.,  4., nan,  2.])

In [51]:
X, y = df.drop(columns=['employee_id', 'is_promoted']), df['is_promoted']

In [52]:
from sklearn.impute import SimpleImputer
edu_imp = SimpleImputer(strategy='constant', fill_value='unknown')
rating_imp = SimpleImputer(strategy='median')

from sklearn.compose import make_column_transformer, make_column_selector
ct = make_column_transformer(
    (edu_imp, ['education']),
    (rating_imp, ['previous_year_rating']),
    verbose_feature_names_out=False
).set_output(transform='pandas')

# X = ct.fit_transform(X)

In [32]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
ct_ohe = make_column_transformer(
    (ohe, make_column_selector(dtype_include=object)),
    ('passthrough', make_column_selector(dtype_exclude=object)),
    verbose_feature_names_out=False
).set_output(transform='pandas')

# X = ct_ohe.fit_transform(X)

In [41]:
X_test = pd.read_csv('HR Analytics/test.csv')
X_test = ct.transform(X_test)
# X_test = ct_ohe.transform(X_test)

In [54]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(random_state=24,
                        cat_features = ['department', 'region', 'education', 'gender', 'recruitment_channel']
                       )
from sklearn.pipeline import Pipeline
pipe = Pipeline(
    [
        ('Imp', ct),
        ('cat', cat)
    ]
)

pipe.fit(X, y)
# y_pred = pipe.predict(X_test)

ValueError: 'department' is not in list

In [26]:
submit = pd.read_csv('HR Analytics/sample_submission.csv')
submit['is_promoted'] = y_pred
submit.to_csv('submit.csv', index=False)